# Telco Classification Project - Draft Workspace
**Please see this repository for JupyterNB styling (for presentation):**
* https://github.com/faithkane3/example_classification_project

## Overview of work done so far...
- Store contents of telco_churn database to local machine
- acquire.py
- prepare.py
- Initial exploration
- Feature selection

## To-Do
- ~~Write acquire.py and prepare.py~~
- ~~Explore data~~
- Modeling/Evaluation
- Create polished JupyterNB for presentation
- Solidify all deliverables
- Practice presentation to peer group
- Deliver!

In [1]:
### Imports ###
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

from acquire import gen_telco
from prepare import gen_prep_telco
from explore import train_validate_test_split
from explore import explore_univariate, explore_bivariate, explore_multivariate

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
### Save local copies of the four telco_churn tables to current directory ###
# gen_telco()

In [3]:
### Prepare telco_churn tables ###
telco, contract_types, internet_service_types, payment_types = gen_prep_telco()
telco.head(3)

,partner,dependents,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,churn,gender,senior_citizen,internet_service_type_id,contract_type_id,payment_type_id,tenure,monthly_charges,total_charges
0,1,1,1,0,0,1,0,1,1,0,1,0,1,0,1,2,2,9,65.6,593.3
1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,1,2,9,59.9,542.4
2,0,0,1,0,0,0,1,0,0,0,1,1,0,0,2,1,1,4,73.9,280.85


In [4]:
### Split customer data ###
train, validate, test = train_validate_test_split(telco, 'churn', seed=123)
train.shape, validate.shape, test.shape

((3943, 20), (1691, 20), (1409, 20))

In [5]:
### Identify categorical and quantified variables, identify target ###
cat_vars = ['partner', 'dependents', 'phone_service', 'multiple_lines',
            'online_security', 'online_backup', 'device_protection', 
            'tech_support', 'paperless_billing', 'streaming_tv', 
            'streaming_movies', 'gender', 'senior_citizen', 
            'internet_service_type_id', 'contract_type_id', 'payment_type_id']

quant_vars = ['tenure', 'monthly_charges', 'total_charges']

target = 'churn'

In [6]:
### Explore! ###
# explore_univariate(train, cat_vars, quant_vars)
# explore_bivariate(train, target, cat_vars, quant_vars)

### Exploration Results:
Note: percentages are estimated from visual checks of bar graphs
- churn: 2897 not churned | 1046 churned
- partner no/yes: 49/51, churn: 33% 19%
- dependents no/yes: 70/30, churn: 31% 15%
- phone_service no/yes: 9/91, churn: same
- multiple_lines no/yes/nophone: 9/48/43, same
- online_security no/yes/nointernet: 29/49/22, churn: 41% 16% 9%
- online_backup no/yes/nointernet: 35/43/22, churn: 40% 21% 8%
- device_protection no/yes/nointernet: 36/43/22, churn: 40% 22% 8%
- tech_support no/yes/nointernet: 29/49/22, churn: 41% 16% 9%
- paperless_billing no/yes: 41/59, churn: 17% 34%
- streaming_tv no/yes/nointernet: 39/39/22, churn: 33% 29% 8%
- streaming_movies no/yes/nointernet: 39/40/22, churn: 33% 30% 8%
- gender no/yes: 49/51, churn: same
- senior_citizen no/yes: 84/16, churn: 25% 40%
- internet_service_type_id DSL/Fiber/None: 34/44/22, churn: 19% 40% 7%
- contract_type_id M2M/1yr/2yr: 55/23/22, churn: 42% 11% 3%
- payment_type_id eCheck/mailed/auto-bank/auto-credit: 34/22/22/22, churn: 46% 22% 18% 18%
- tenure: highest at minimum, high at maximum (U-shaped curve)
- monthly_charges: peak at lowest, normal distribution centered around 80
- total_charges: nearly-uniform (slightly skews right)

Features with a value having high churn rate (>40%):
- online_security: no, 30% of all telco customers
- online_backup: no, 35% of all telco
- device_protection: no, 36% of population
- tech_support: no, 29%pop
- senior_citizen: yes, 16%pop
- internet_service_type_id: Fiber optic, 44%pop
- contract_type_id: Month-to-month, 55%pop
- payment_type_id: electronic check, 34%pop

Features with a value having moderately-high churn rate (>30%):
- partner: no, 49%pop
- dependents: no, 70%pop
- paperless_billing: yes, 59%pop
- streaming_tv: no, 39%
- streaming_movies: no/yes, 39%/40%

Numeric features:
- tenure, monthly_charges, total_charges

Features with similar churn rate between values (discard):
- phone_service, multiple_lines, gender


### Candidates (Requires further analysis):
- Partner
- Dependents
- InternetService
- OnlineBackup
- DeviceProtection
- TechSupport
- Contract
- PaperlessBilling
- PaymentMethod

Not Candidates (and why):
- Gender: No significant difference in churn rate
- PhoneService: No significant difference in churn rate
- MultipleLines: Not a very significant difference in churn rate
- Senior Citizen: Small population compared to total

In [7]:
### In-Common Populations ###

# Goal: For a few features whose high-churning value is 
#       between 30% and 60% of total population,
#       determine if any two have too many values in common to be added to the model.

# Features to investigate: partner(Single), internet_service_type_id(Fiber), online_backup(Nobackup), 
#                          device_protection(Unprotected), contract_type_id(Month-to-month)

fs = (train[train.internet_service_type_id == 2].partner == 0).mean()
fn = (train[train.internet_service_type_id == 2].online_backup == 0).mean()
fu = (train[train.internet_service_type_id == 2].device_protection == 0).mean()
fm = (train[train.internet_service_type_id == 2].contract_type_id == 1).mean()

sn = (train[train.partner == 0].online_backup == 0).mean()
su = (train[train.partner == 0].device_protection == 0).mean()
sm = (train[train.partner == 0].contract_type_id == 1).mean()

nu = (train[train.online_backup == 0].device_protection == 0).mean()
nm = (train[train.online_backup == 0].contract_type_id == 1).mean()

mu = (train[train.contract_type_id == 1].device_protection == 0).mean()

# Print percent-in-common for each two-feature combination
print(fs, fn, fu, fm, sn, su, sm, nu, nm, mu)

0.5117478510028654 0.5547277936962751 0.5461318051575931 0.6836676217765043 0.49604743083003955 0.5004940711462451 0.6902173913043478 0.6345475910693302 0.7391304347826086 0.6064516129032258


Commonality:
- 40s% Share: Single-Nobackup (49.6%)
- 50s% Share: Single-Unprotected (50.1%), Fiber-Single (51.2%), Fiber-Unprotected (54.6%), Fiber-Nobackup (55.5%)
- 60s% Share: Monthly-Unprotected (60.6%), Nobackup-Unprotected (63.45%), Fiber-Monthly (68.4%), Single_Monthly (69.0%)
- 70s% Share: Nobackup-Monthly (73.9%)

### Takeaways: 
- **Although Month-to-month contracts are 55% of all contracts with a significantly-higher churn rate than other contracts, the population comparison above shows that month-to-month adds population redundancy. Therefore, I will create a model including month-to-month and a model not including month-to-month to see which model performs better.**
- **The only other population check with >60% share between variables is device_protection and online_backup. I will therefore drop one for a model, and drop the other for another model.**

### Model Design
Model Combination #1 Target and Features:
- Target: churn
- Features: partner, internet_service_type_id, online_backup

Model Combination #2 Target and Features:
- Target: churn
- Features: partner, internet_service_type_id, online_backup, contract_type_id

Model Combination #3 Target and Features:
- Target: churn
- Features: partner, internet_service_type_id, device_protection

In [8]:
target = 'churn'
combo1 = ['partner', 'internet_service_type_id', 'online_backup']
combo2 = ['partner', 'internet_service_type_id', 'online_backup', 'contract_type_id']
combo3 = ['partner', 'internet_service_type_id', 'device_protection']

y_train = train[target]
X_train1 = train[combo1]
X_train2 = train[combo2]
X_train3 = train[combo3]

y_validate = validate[target]
X_validate1 = validate[combo1]
X_validate2 = validate[combo2]
X_validate3 = validate[combo3]

X_train2.head(3)

,partner,internet_service_type_id,online_backup,contract_type_id
5310,1,1,1,3
3790,1,1,1,3
4398,1,2,0,2


### Models
I will use Random Forest, K-Nearest Neighbors, and Logistic Regression for each model combination listed above.

In [9]:
rf1 = RandomForestClassifier(min_samples_leaf=1, max_depth=10, random_state=123)
rf2 = RandomForestClassifier(min_samples_leaf=1, max_depth=10, random_state=123)
rf3 = RandomForestClassifier(min_samples_leaf=1, max_depth=10, random_state=123)

knn1 = KNeighborsClassifier(n_neighbors=10)
knn2 = KNeighborsClassifier(n_neighbors=10)
knn3 = KNeighborsClassifier(n_neighbors=10)

logit1 = LogisticRegression(random_state=123)
logit2 = LogisticRegression(random_state=123)
logit3 = LogisticRegression(random_state=123)

In [10]:
rf1 = rf1.fit(X_train1, y_train)
rf2 = rf2.fit(X_train2, y_train)
rf3 = rf3.fit(X_train3, y_train)

knn1 = knn1.fit(X_train1, y_train)
knn2 = knn2.fit(X_train2, y_train)
knn3 = knn3.fit(X_train3, y_train)

logit1 = logit1.fit(X_train1, y_train)
logit2 = logit2.fit(X_train2, y_train)
logit3 = logit3.fit(X_train3, y_train)

### Evaluation
Checking baseline.

In [11]:
# train.groupby('churn').churn.count()
(train.churn == 0).mean()

0.7347197565305605

Checking scores of each model against train.

In [12]:
print(rf1.score(X_train1, y_train))
print(rf2.score(X_train2, y_train))
print(rf3.score(X_train3, y_train))

print(knn1.score(X_train1, y_train))
print(knn2.score(X_train2, y_train))
print(knn3.score(X_train3, y_train))

print(logit1.score(X_train1, y_train))
print(logit2.score(X_train2, y_train))
print(logit3.score(X_train3, y_train))

0.7550088764899823
0.7702257164595485
0.7514582804970834
0.7347197565305605
0.7159523205680953
0.7347197565305605
0.7550088764899823
0.7702257164595485
0.7514582804970834


Checking model scores against validate.

In [13]:
print(rf1.score(X_validate1, y_validate))
print(rf2.score(X_validate2, y_validate))
print(rf3.score(X_validate3, y_validate))

print(knn1.score(X_validate1, y_validate))
print(knn2.score(X_validate2, y_validate))
print(knn3.score(X_validate3, y_validate))

print(logit1.score(X_validate1, y_validate))
print(logit2.score(X_validate2, y_validate))
print(logit3.score(X_validate3, y_validate))

0.7480780603193377
0.7640449438202247
0.7474866942637493
0.7344766410408042
0.7261975162625666
0.7344766410408042
0.7480780603193377
0.7640449438202247
0.7474866942637493


#### Baseline
- Not-churned customers make up 73.5% of all telco customers.

#### Train results
- rf2 and logit2 have same score, 77.0%
- rf1 and logit1 have same score, 75.5%
- rf3 and logit3 have same score, 75.1%
- knn1 and knn3 have same score, 73.5%
- knn2 scored lowest with 71.6%

#### Validate results
- rf2 and logit2 have same score, 76.4%
- rf1 and logit1 have same score, 74.8%
- rf3 and logit3 have same score, 74.7%
- knn1 and knn3 have same score, 73.4%
- knn2 scored lowest with 72.6%

### Evaluation Takeaways
- Random Forest (RF) and Logistic Regression (LOGIT) score higher than baseline, K-Nearest Neighbors scores lower
- Including the contract_type_id feature improved RF and LOGIT performance for this evaluation